# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import sys

from azureml.core import Environment
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.workspace import Workspace
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails


## Setup Workspace and compute cluster

In [2]:
# Worspace and experiment

ws = Workspace.from_config()
experiment_name = 'automl-experiment'
project_folder = '.'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: aml-wwe-ictx-dsplay
Azure region: westeurope
Subscription id: ec5ba19e-6205-418f-a52d-d0943090ca16
Resource group: rg-wwe-ictx-dsplayground


In [3]:
# Create compute cluster

compute_cluster_name = "alpha"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, please use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, please use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Dataset

dataset_name = "HrAnalytics-DataScientist"

if dataset_name in ws.datasets.keys():
        dataset = ws.datasets[dataset_name]

else:
        sys.path.append("..")
        from scripts.create_dataset import main as register_dataset
        # Create AML Dataset and register it into Workspace
        register_dataset()
        dataset = ws.datasets[dataset_name]


df = dataset.to_pandas_dataframe()
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,None,None,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,None,No relevent experience,Full time course,Graduate,STEM,5,None,None,never,83,0.0
3,33241,city_115,0.789,None,No relevent experience,None,Graduate,Business Degree,<1,None,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


## Data Preparation Step



In [5]:

env = Environment('dataprep')

conda = CondaDependencies()

# add pip packages
conda.add_pip_package('azureml-core')
conda.add_pip_package('pandas')
conda.add_pip_package('numpy')
conda.add_pip_package("sklearn")

# add conda dependencies to environment
env.python.conda_dependencies = conda

runconfig = RunConfiguration()
runconfig.environment  = env
runconfig.target = compute_cluster





In [12]:



prepped_data = PipelineData(name="prepared_data").as_dataset()

dataprep_step = PythonScriptStep(
    name="dataprep", 
    script_name="dataprep.py",
    source_directory="./steps_scripts/prep",
    compute_target=ComputeTarget(ws,name="lancia"), 
    arguments=["--output_path", prepped_data],
    inputs=[dataset.as_named_input('input_ds')],
    outputs=[prepped_data],
    runconfig=runconfig,
    allow_reuse=True
)


# Train Test Split

In [13]:


train_data = PipelineData(name="train_data").as_dataset()
test_data =  PipelineData(name="test_data").as_dataset()

split_step = PythonScriptStep(
    name="train_test_split", 
    script_name="train_test_split.py", 
    source_directory="./steps_scripts/split/",
    compute_target=ComputeTarget(ws,name="lancia"), 
    arguments=["--train_path", train_data, "--test_path", test_data],
    inputs=[prepped_data.parse_delimited_files().as_named_input("prepped_data")],
    outputs=[train_data, test_data],
    runconfig=runconfig,
    allow_reuse=True
)



## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:

from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [15]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 4,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_cluster,
                             task="classification",
                             label_column_name="target",   
                             path=project_folder,
                             training_data =train_data.parse_delimited_files(),
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log="automl_errors.log",
                             **automl_settings
                            )

from azureml.pipeline.steps import AutoMLStep

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)



In [16]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[dataprep_step,split_step,automl_step])

In [18]:
remote_run = experiment.submit(pipeline)

Created step dataprep [ab98cf3e][b152e88b-97f3-423d-88d0-6cab2c73928d], (This step is eligible to reuse a previous run's output)
Created step train_test_split [d3e02119][8912289d-6bb6-43f1-9f1e-eb76442432c1], (This step is eligible to reuse a previous run's output)
Created step automl_module [130530dd][7371a1f6-036e-4c28-b672-da234ec464f9], (This step will run and generate new outputs)
Submitted PipelineRun 40e6474a-5477-4fa9-8035-06e10f1089c9
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/40e6474a-5477-4fa9-8035-06e10f1089c9?wsid=/subscriptions/ec5ba19e-6205-418f-a52d-d0943090ca16/resourcegroups/rg-wwe-ictx-dsplayground/workspaces/aml-wwe-ictx-dsplay&tid=c16e514b-893e-4a01-9a30-b8fef514a650


## Run Details


In [40]:
from azureml.train.automl.run import AutoMLRun

a = remote_run.find_step_run("automl_module")[0]
a = AutoMLRun(experiment, a.id)
type(a)
a._type = "azureml.AutoMLRun"

AttributeError: can't set attribute

In [31]:
from azureml.pipeline.steps import AutoMLStepRun
RunDetails(AutoMLStepRun(remote_run.find_step_run("automl_module")[0])).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:

remote_run.wait_for_completion(show_output=False)


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [41]:
from azureml.train.automl.run import AutoMLRun


def get_module_runid(pipe_run, step_name)
    for step in pipe_run.get_steps():
        step_run_id = step.id
        if step.name == step_name:
            print(step_run_id)
            break
    else:
        raise KeyError("missing step", step_name)
        
    return step_run_id

automl_run = AutoMLRun(experiment = experiment, run_id=get_module_runid(remote_run, "automl_module"))
#RunDetails(automl_run).show()

SyntaxError: invalid syntax (<ipython-input-41-40c53be5335b>, line 4)

In [21]:
best_run_automl, best_model_automl = automl_run.get_output()
best_run_metrics_automl = best_run_automl.get_metrics()

print("\n")
print('Best run ID: ', best_run_automl.id)
print('Best run Accuracy: ', best_run_metrics_automl['accuracy'])
print('Metrics: ', best_run_metrics_automl)
print("\n")



Best run ID:  ff682bcd-529d-4e90-8776-e72c31498d95_28
Best run Accuracy:  0.7927701283453465
Metrics:  {'AUC_macro': 0.8027063976956555, 'f1_score_macro': 0.7156989468962986, 'log_loss': 0.46049461248784, 'average_precision_score_macro': 0.7256667425098365, 'AUC_micro': 0.8762532368995313, 'f1_score_micro': 0.7927701283453465, 'precision_score_macro': 0.7221505732794168, 'matthews_correlation': 0.43229184940574455, 'balanced_accuracy': 0.7103057230882517, 'accuracy': 0.7927701283453465, 'recall_score_micro': 0.7927701283453465, 'precision_score_micro': 0.7927701283453465, 'average_precision_score_micro': 0.8650132427660608, 'norm_macro_recall': 0.4206114461765035, 'average_precision_score_weighted': 0.8167048754781403, 'weighted_accuracy': 0.8420746587666881, 'precision_score_weighted': 0.7877432684137163, 'recall_score_macro': 0.7103057230882517, 'recall_score_weighted': 0.7927701283453465, 'f1_score_weighted': 0.7899236938044379, 'AUC_weighted': 0.8027063976956555, 'accuracy_table'

In [22]:
print(best_model_automl)


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.056842105263157895, min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False))], flatten_transform=None, weights=[0.07142857142857142, 0.35714285714285715, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.0714285714

In [26]:
import joblib
os.makedirs('outputs', exist_ok=True)
joblib.dump(best_model_automl, filename='outputs/best_model_automl.pkl')

['outputs/best_model_automl.pkl']

# Test Model

In [45]:
import pandas as pd 
import glob


# functions to download output to local and fetch as dataframe
def get_download_path(download_path, output_name):
    output_folder = os.listdir(download_path + '/azureml')[0]
    path =  download_path + '/azureml/' + output_folder + '/' + output_name
    return path

def fetch_df(current_step, output_name):
    output_data = current_step.get_output_data(output_name)    
    download_path = './outputs/' + output_name
    output_data.download(download_path, overwrite=True)
    df_path = get_download_path(download_path, output_name) 
    all_files = glob.glob(df_path + "/*.csv")
    print(all_files)
    df_from_each_file = (pd.read_csv(f) for f in all_files)
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    
 
    return concatenated_df

In [46]:
split_step = remote_run.find_step_run(split_step.name)[0]
test_df = fetch_df(split_step, test_data.name)



['./outputs/test_data/azureml/7c6f5359-70b8-4905-aca9-b81086805c51/test_data/test.csv']


In [48]:
y_true = test_df["target"]
x = test_df.drop(columns=["target"])

In [49]:
y_pred= best_model_automl.predict(x)


In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true,y_pred)

0.7745302713987474

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service